In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from boids_quantative_analysis import get_velocities, calculate_order_parameter

In [2]:
class Scene:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    

    def init_swarm(self, number_of_particles: int):
        swarm = []
        for _ in np.arange(number_of_particles):
            swarm.append([[np.random.uniform(0,self.width), np.random.uniform(0,self.height)],[np.random.uniform(-1,1), np.random.uniform(-1,1)]])
        return np.array(swarm)

    
    def get_neighbors(self, particle, swarm, interaction_radius):
        return swarm[np.linalg.norm(swarm[:,0] - particle[0], axis=1) <= interaction_radius]

In [3]:
def wrap_particle(particle, scene_width, scene_height):
    if particle[0,0] < 0:
        particle[0,0] += scene_width
    if particle[0,0] > scene_width:
        particle[0,0] -= scene_width
    if particle[0,1] < 0:
        particle[0,1] += scene_height
    if particle[0,1] > scene_height:
        particle[0,1] -= scene_height
    return particle


# Each particle has the following structure:
#   [[x,y],2d_direction_vector]


def update_particle(particle, neighbors, cohesion_factor, alignment, separation_factor, velocity):
    avg_angle = np.arctan2(particle[0,1], particle[0,0])
    avg_position = particle[0]
    avg_distance = np.array([0,0])
    if neighbors.shape[0] > 1:
        avg_position = np.mean(neighbors[:,0],axis=0)
        avg_angle = np.arctan2(np.mean(neighbors[:,0,1]), np.mean(neighbors[:,0,0]))
        norm_sq = np.linalg.norm(particle[0] - neighbors[:,0], axis=1) ** 2
        particle_index = np.argmin(norm_sq)
        neighbors, norm_sq = np.delete(neighbors, particle_index, axis=0), np.delete(norm_sq, particle_index, axis=0)
        avg_distance = np.sum((particle[0] - neighbors[:,0]) / np.array([norm_sq,norm_sq]).T, axis=0) / (neighbors.shape[0] + 1)
    avg_angle += (np.random.random() * alignment) - 0.25
    particle[1,0], particle[1,1] = np.cos(avg_angle), np.sin(avg_angle)
    cohesion = (avg_position - particle[0]) / cohesion_factor
    particle[1] += cohesion
    avg_distance *= separation_factor
    particle[1] += avg_distance
    particle[1] *= velocity
    particle[0] += particle[1]
    return particle

In [ ]:
def plot_swarm(swarm):
    plt.scatter(x=swarm[:,0,0],y=swarm[:,0,1])
    plt.show()

def simulate(scene_width, scene_height, particles_per_simulation, cohesion_factor, alignment, separation_factor, velocity, interaction_radius, num_steps):
    scene = Scene(scene_width,scene_height)
    swarm = scene.init_swarm(particles_per_simulation)
    result = {'id':[], 'x':[], 'y':[], 'iteration':[]}
    for iteration in np.arange(num_steps):
        # plot_swarm(swarm)
        for i, particle in enumerate(swarm):
            # print(f'{i}, {particle[0]}, {particle[1]}, {iteration}')
            result['id'].append(i)
            result['x'].append(particle[0,0])
            result['y'].append(particle[0,1])
            result['iteration'].append(iteration)
            neighbors = scene.get_neighbors(particle, swarm, interaction_radius)
            particle = update_particle(particle, neighbors, cohesion_factor, alignment, separation_factor, velocity)
            swarm[i] = wrap_particle(particle, scene.width, scene.height)
    return pd.DataFrame.from_dict(result)

# csv_content = simulate()
# with open('test_1.csv', 'w') as csv:
#     for line in csv_content:
#         csv.write(line+'\n')

In [ ]:
# Define constants
N = 20  # Number of accepted Boid simulation parameters
particles_per_simulation = 15
num_steps = 300

# Define fixed parameters
velocity = 1
interaction_radius = 10
exclusion_radius = 2
field_size = 100 # Both width and heigh

prior_mean = np.array([15, 0.5, 3])  # Mean of the prior distribution
prior_std = np.array([1, 0.3, 0.5])  # Standard deviation of the prior distribution

epsilon_tau = 0.1 


def generate_candidates(accepted_parameters):
    candidates = []
    for parameters in accepted_parameters:
        perturbed_parameters = np.random.normal(parameters, prior_std)
        candidates.append(perturbed_parameters)
    return candidates


def initialize_population():
    population = []
    for _ in range(N):
        cohesion = np.random.normal(prior_mean[0], prior_std[0])
        alignment = np.random.normal(prior_mean[1], prior_std[1])
        separation = np.random.normal(prior_mean[2], prior_std[2])
        population.append((cohesion, alignment, separation))
    return population


def run_simulation(parameters):
    order_parameters = []
    for cohesion, alignment, separation in parameters:
        # Run Boid simulation with given parameters
        simulation_results = simulate(field_size,field_size,particles_per_simulation,cohesion,alignment,separation,velocity,interaction_radius,num_steps)
        # Calculate order parameter for the simulation
        velocities = get_velocities(simulation_results)
        order_parameter = calculate_order_parameter(velocities)
        order_parameters.append(order_parameter)
    return order_parameters


# Define function to perform ABC iterations
def perform_iterations(population):
    accepted_parameters = []
    for _ in range(num_iterations):
        candidate_parameters = generate_candidates(accepted_parameters)
        candidate_order_parameters = run_simulation(candidate_parameters)
        accepted_parameters = accept_reject(candidate_parameters, candidate_order_parameters)
        save_accepted_parameters(accepted_parameters)


# Define function to generate new candidate parameters
def generate_candidates(accepted_parameters):
    # Generate new candidate parameters by perturbing the accepted parameters
    # Use a normal distribution with a certain standard deviation for perturbation
    candidates = []
    for parameters in accepted_parameters:
        perturbed_parameters = np.random.normal(parameters, prior_std)
        candidates.append(perturbed_parameters)
    return candidates


# Define function to accept or reject candidate parameters based on order parameter
def accept_reject(candidate_parameters, candidate_order_parameters):
    accepted_parameters = []
    for i in range(len(candidate_parameters)):
        if candidate_order_parameters[i] >= epsilon_tau:
            accepted_parameters.append(candidate_parameters[i])
    return accepted_parameters



def save_accepted_parameters(accepted_parameters):
    # Save accepted parameters to a file or data structure for later analysis
    pass

# Main function
def abc():
    population = initialize_population()
    perform_iterations(population)
